In [131]:
import sympy as sp
from IPython.display import display, Math

In [190]:
def generatePoly (n):
    
    symbols = []
    for i in range(n+1):
      symbols.append(sp.symbols(f"a_{i}"))
    
    s = sp.symbols('s')
    result = sp.poly(0, s)
    for x in range(n+1):
        result = result.add(sp.poly(symbols[x]*s**x,s))
    return result, symbols

In [191]:
w, s = sp.symbols('w s')

In [192]:
a, b, c, k = sp.symbols('a b c k', positive=True)

In [193]:
H, f, Dem, p, np, result = sp.symbols('H, f, Dem p np result', cls=sp.Function)

Dada la función:

$$\left|H(s)\right|^2 = \frac{1}{14s^4+4s^2+2}$$

Lo podríamos separar en:

$$H(s)\cdot H(−s) = \frac{1}{14s^4+4s^2+2} = \frac{1}{a_1s^2 + a_0s + a} \cdot \frac{1}{a_1s^2 - a_0s + a}$$

Esto lo podemos generalizar para cualquier polinomio de grado n par. Primero ponemos el polinomio de $|H(s)|^2$ en una variable de sympy de tipo poly

In [202]:
Dem = sp.poly(4*s**4 + 4*s**2 + 2, s) 
Dem

Poly(4*s**4 + 4*s**2 + 2, s, domain='ZZ')

Luego generamos los polinomios genéricos pra $H(s)$ y $H(-s)$

In [253]:
p2, symbols = generatePoly(2)
np2 = sp.poly(p2.subs({s:-s}))
print(display(Math(sp.latex(p2))))
print(display(Math(sp.latex(np2))))

<IPython.core.display.Math object>

None


<IPython.core.display.Math object>

None


Multiplicamos ambos polinomios y guardamos en un nueva variable:

In [211]:
result = sp.poly(p2 * np2, s)
result

Poly(a_2**2*s**4 + (2*a_0*a_2 - a_1**2)*s**2 + a_0**2, s, domain='ZZ[a_0,a_1,a_2]')

Ahora obtendremos los coeficientes, para ello necesitamos un array con todo nuestro sistema de ecuaciones igualado a $0$, lo obtenemos de la siguiente forma:

In [214]:
def coefArray (poly, dem):
    result = []
    n = sp.degree(dem)
    for i in range(n+1):
        result.append(poly.coeff_monomial(s**i) - dem.coeff_monomial(s**i))
    return result

In [241]:
coefarray = coefArray(result, Dem)
for x in coefarray:
    equation = display(Math(sp.latex(x)))
    if equation is not None:
        print(equation)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Solucionamos el sistema con nonlinsolve y listamos sólo aquellas donde todos los coeficientes sean positivos:

In [254]:
soluciones = sp.nonlinsolve(coefarray, symbols)

In [255]:
def hasProperty(sol):
    if(sol[0].is_real and sol[1].is_real and sol[2].is_real):
        return sp.GreaterThan(sol[0], 0) and sp.GreaterThan(sol[1], 0) and sp.GreaterThan(sol[2], 0)

In [256]:
finalResult = list(filter(hasProperty, soluciones))[0]
finalResult

(sqrt(2), sqrt(-4 + 4*sqrt(2)), 2)

Reemplazamos en nuestro polinomio de $H(s)$ positivo:

In [264]:
H = p2.subs({symbols[0]:finalResult[0], symbols[1]:finalResult[1], symbols[2]:finalResult[2]})
H = 1/H
H

1/(2*s**2 + s*sqrt(-4 + 4*sqrt(2)) + sqrt(2))